In [3]:
# Import modulos necesarios
import pandas as pd
import json
import os
import requests
import time
import unicodedata as ud

In [27]:
"""
Abrimos el archivo Partidos.csv en un dataframe para su tratamiento
"""
cols = ["EquipoAnf", "EquipoVis", "ResulAnf", "ResulVis", "Fecha","Estadio", "Ciudad", "Arbitro"]
dsFutbol = pd.read_csv("Partidos.csv", sep=',', header=None, names=cols)
dsFutbol

,EquipoAnf,EquipoVis,ResulAnf,ResulVis,Fecha,Estadio,Ciudad,Arbitro
0,Elche,Real Sociedad,0,3,26/09/2020,Estadio Martínez Valero,Elche,Díaz de Mera
1,Real Sociedad,Real Madrid,0,0,20/09/2020,Anoeta,San Sebastián,Martínez Munuera
2,Eibar,Osasuna,0,0,18/10/2020,Estadio Municipal de Ipurua,n/d,Medié Jiménez
3,Barcelona,Real Madrid,1,3,24/10/2020,Camp Nou,Barcelona,Martínez Munuera
4,Eibar,Athletic Club,1,2,27/09/2020,Estadio Municipal de Ipurua,n/d,Cordero Vega
...,...,...,...,...,...,...,...,...
7668,Barcelona,Espanyol,4,2,10/12/2000,Nou Camp (Nou Camp),Barcelona,Daudén Ibáñez
7669,Real Sociedad,Celta de Vigo,2,2,04/02/2001,Anoeta,San Sebastián,Andradas Asurmendi
7670,Numancia,Barcelona,1,1,18/03/2001,Los Pajaritos (Los Pajaritos),Soria,Pérez Burrull
7671,Real Madrid,Racing de Santander,1,0,17/02/2001,Santiago Bernabéu,Madrid,Daudén Ibáñez


In [28]:
# Comprobamos la existencia de registros con la columna "ciudad" sin informar
df = dsFutbol[dsFutbol['Ciudad'] == 'n/d']
df['EquipoAnf'].unique()

array(['Eibar', 'Xerez'], dtype=object)

In [29]:
"""
Actualizamos el campo Ciudad para que aquellos equipos que tenga asociado a la columna 'Ciudad' el valor 'n/d'
"""
# @Ramón, he modificado la forma en la que actualizabamos el dataframe para evitar el warning por SettingWithCopyWarning
dsFutbol.loc[dsFutbol['EquipoAnf'] == 'Eibar','Ciudad'] = 'Eibar' 
dsFutbol.loc[dsFutbol['EquipoAnf'] == 'Xerez','Ciudad'] = 'Jerez de la Frontera' 

dsFutbol.head()

,EquipoAnf,EquipoVis,ResulAnf,ResulVis,Fecha,Estadio,Ciudad,Arbitro
0,Elche,Real Sociedad,0,3,26/09/2020,Estadio Martínez Valero,Elche,Díaz de Mera
1,Real Sociedad,Real Madrid,0,0,20/09/2020,Anoeta,San Sebastián,Martínez Munuera
2,Eibar,Osasuna,0,0,18/10/2020,Estadio Municipal de Ipurua,Eibar,Medié Jiménez
3,Barcelona,Real Madrid,1,3,24/10/2020,Camp Nou,Barcelona,Martínez Munuera
4,Eibar,Athletic Club,1,2,27/09/2020,Estadio Municipal de Ipurua,Eibar,Cordero Vega


In [30]:
"""
Se carga el dataset de municipios
"""
dsMunicipios = pd.read_excel('11codmun.xls', skiprows=1)
dsMunicipios

,CPRO,CMUN,DC,NOMBRE
0,1,1,4,Alegría-Dulantzi
1,1,2,9,Amurrio
2,1,49,3,Añana
3,1,3,5,Aramaio
4,1,6,6,Armiñón
...,...,...,...,...
8111,50,296,7,"Zaida, La"
8112,50,297,3,Zaragoza
8113,50,298,9,Zuera
8114,51,1,3,Ceuta


In [31]:
"""
Se hace 'merge' de los datos de los partidos con el de minucipios como paso previo a obtener las provincias donde
están ubicados los estadios
"""
result = pd.merge(dsFutbol, dsMunicipios, left_on="Ciudad", right_on="NOMBRE", how='left')
result.head()

,EquipoAnf,EquipoVis,ResulAnf,ResulVis,Fecha,Estadio,Ciudad,Arbitro,CPRO,CMUN,DC,NOMBRE
0,Elche,Real Sociedad,0,3,26/09/2020,Estadio Martínez Valero,Elche,Díaz de Mera,3,65,1,Elche
1,Real Sociedad,Real Madrid,0,0,20/09/2020,Anoeta,San Sebastián,Martínez Munuera,20,69,7,San Sebastián
2,Eibar,Osasuna,0,0,18/10/2020,Estadio Municipal de Ipurua,Eibar,Medié Jiménez,20,30,0,Eibar
3,Barcelona,Real Madrid,1,3,24/10/2020,Camp Nou,Barcelona,Martínez Munuera,8,19,3,Barcelona
4,Eibar,Athletic Club,1,2,27/09/2020,Estadio Municipal de Ipurua,Eibar,Cordero Vega,20,30,0,Eibar


In [32]:
"""
Comprobamos la existencia de registros con la columna "NOMBRE" sin informar
"""
df = result[result['NOMBRE'].isnull()]
df['EquipoAnf'].unique()

array([], dtype=object)

In [34]:
"""
Recuperamos en un dataframe(df) los datos de provincias para hacer 'matching' con el df generado en pasos anteriores
(result)
"""
dsProvincias = pd.read_csv("provinces_es.csv", sep=',')
dsProvincias.head()

,code,postal_code,name,phone_code,iso2
0,VI,1,Álava,945,ES
1,AB,2,Albacete,967,ES
2,A,3,Alacant,950,ES
3,AL,4,Almería,920,ES
4,AV,5,Ávila,924,ES


In [35]:
"""
Se hace 'merge' de los datos de los partidos con el de minucipios como paso previo a obtener las provincias donde
están ubicados los estadios
"""
result2 = pd.merge(result, dsProvincias, left_on="CPRO", right_on="postal_code", how='left')
result2

,EquipoAnf,EquipoVis,ResulAnf,ResulVis,Fecha,Estadio,Ciudad,Arbitro,CPRO,CMUN,DC,NOMBRE,code,postal_code,name,phone_code,iso2
0,Elche,Real Sociedad,0,3,26/09/2020,Estadio Martínez Valero,Elche,Díaz de Mera,3,65,1,Elche,A,3,Alacant,950,ES
1,Real Sociedad,Real Madrid,0,0,20/09/2020,Anoeta,San Sebastián,Martínez Munuera,20,69,7,San Sebastián,SS,20,Gipuzkoa,943,ES
2,Eibar,Osasuna,0,0,18/10/2020,Estadio Municipal de Ipurua,Eibar,Medié Jiménez,20,30,0,Eibar,SS,20,Gipuzkoa,943,ES
3,Barcelona,Real Madrid,1,3,24/10/2020,Camp Nou,Barcelona,Martínez Munuera,8,19,3,Barcelona,B,8,Barcelona,93,ES
4,Eibar,Athletic Club,1,2,27/09/2020,Estadio Municipal de Ipurua,Eibar,Cordero Vega,20,30,0,Eibar,SS,20,Gipuzkoa,943,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7668,Barcelona,Espanyol,4,2,10/12/2000,Nou Camp (Nou Camp),Barcelona,Daudén Ibáñez,8,19,3,Barcelona,B,8,Barcelona,93,ES
7669,Real Sociedad,Celta de Vigo,2,2,04/02/2001,Anoeta,San Sebastián,Andradas Asurmendi,20,69,7,San Sebastián,SS,20,Gipuzkoa,943,ES
7670,Numancia,Barcelona,1,1,18/03/2001,Los Pajaritos (Los Pajaritos),Soria,Pérez Burrull,42,173,6,Soria,SO,42,Soria,975,ES
7671,Real Madrid,Racing de Santander,1,0,17/02/2001,Santiago Bernabéu,Madrid,Daudén Ibáñez,28,79,6,Madrid,M,28,Madrid,91,ES


In [10]:
"""
Eliminamos las columnas del dataframe resultante que no resulten de utilidad
"""
result2.drop(['CPRO', 'CMUN', 'DC', 'NOMBRE', 'code', 'postal_code', 'phone_code', 'iso2'], axis = 'columns', inplace=True)

In [11]:
"""
    Realizamos algunos ajustes sobre el dataframe result2 que contiene los datos de partidos con la provincia anexada
"""

# Cambiamos el nombre de la columns 'nombre' por Provincia, para que la cabecera quede correcta
result2.rename(columns={'name':'Provincia'}, inplace=True)
# result2.columns

# Cambiamos el formato de la columna Fecha a formato fecha para poder ordenar los datos por dicha 
# columna de forma ascendente
result2['Fecha'] = pd.to_datetime(result2['Fecha'], format='%d/%m/%Y')

# Creamos una copia ordenada por fecha del dataframe resulta2
df_Res_date = result2.sort_values(['Fecha','Provincia'])

df_Res_date

,EquipoAnf,EquipoVis,ResulAnf,ResulVis,Fecha,Estadio,Ciudad,Arbitro,Provincia
7637,Deportivo de La Coruña,Athletic Club,2,0,2000-09-09,Nuevo Riazor,La Coruña,Andradas Asurmendi,A Coruña
7314,Barcelona,Málaga,2,1,2000-09-09,Nou Camp (Nou Camp),Barcelona,Mejuto González,Barcelona
7420,Real Sociedad,Racing de Santander,2,2,2000-09-09,Anoeta,San Sebastián,Prados García,Gipuzkoa
7551,Real Madrid,Valencia,2,1,2000-09-09,Santiago Bernabéu,Madrid,Daudén Ibáñez,Madrid
7385,Zaragoza,Espanyol,1,2,2000-09-09,La Romareda,Zaragoza,Turienzo Álvarez,Zaragoza
...,...,...,...,...,...,...,...,...,...
68,Granada,Levante,1,1,2020-11-01,Estadio Nuevo Los Carmenes,Granada,Jaime Latre,Granada
67,Celta de Vigo,Real Sociedad,1,4,2020-11-01,Balaidos,Vigo,Cordero Vega,Pontevedra
23,Betis,Elche,3,1,2020-11-01,Benito Villamarín,Sevilla,Medié Jiménez,Sevilla
31,Valencia,Getafe,2,2,2020-11-01,Camp de Mestalla,Valencia,Figueroa Vázquez,Valéncia


In [12]:
"""
Normalizamos los datos almacenados en la columna 'Provincia' para eliminar tildes y convertir a Mayúsculas
"""
df_Res_date['Provincia'] = df_Res_date['Provincia'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

In [18]:
"""
Dado a que los nombres de algunas provincias no quedaron correctas, ya sea por efecto de la normalización 
previa (A Coruña pasó a ser A CORUÑA), ya sea porque en origen el nombre de la provincia no estaba en español
(Castelló en lugar de Castellón), se procede a corregir dichos nombres para que puedan concrdar con lo almacenado
en el fichero de mediciones
"""
# Actualizamos columna provincia para aplicar las correcciones necesarias
df_Res_date.loc[df_Res_date['Provincia'] == 'A CORUNA','Provincia'] = 'A CORUÑA'
df_Res_date.loc[df_Res_date['Provincia'] == 'CASTELLO','Provincia'] = 'CASTELLON'
df_Res_date.loc[df_Res_date['Provincia'] == 'NAFARROA','Provincia'] = 'NAVARRA'
df_Res_date.loc[df_Res_date['Provincia'] == 'ALAVA','Provincia'] = 'ARABA/ALAVA'
df_Res_date.loc[df_Res_date['Provincia'] == 'ALACANT','Provincia'] = 'ALICANTE'

#  Mostramos el contenido resultante
df_Res_date

,EquipoAnf,EquipoVis,ResulAnf,ResulVis,Fecha,Estadio,Ciudad,Arbitro,Provincia
7637,Deportivo de La Coruña,Athletic Club,2,0,2000-09-09,Nuevo Riazor,La Coruña,Andradas Asurmendi,A CORUÑA
7314,Barcelona,Málaga,2,1,2000-09-09,Nou Camp (Nou Camp),Barcelona,Mejuto González,BARCELONA
7420,Real Sociedad,Racing de Santander,2,2,2000-09-09,Anoeta,San Sebastián,Prados García,GIPUZKOA
7551,Real Madrid,Valencia,2,1,2000-09-09,Santiago Bernabéu,Madrid,Daudén Ibáñez,MADRID
7385,Zaragoza,Espanyol,1,2,2000-09-09,La Romareda,Zaragoza,Turienzo Álvarez,ZARAGOZA
...,...,...,...,...,...,...,...,...,...
68,Granada,Levante,1,1,2020-11-01,Estadio Nuevo Los Carmenes,Granada,Jaime Latre,GRANADA
67,Celta de Vigo,Real Sociedad,1,4,2020-11-01,Balaidos,Vigo,Cordero Vega,PONTEVEDRA
23,Betis,Elche,3,1,2020-11-01,Benito Villamarín,Sevilla,Medié Jiménez,SEVILLA
31,Valencia,Getafe,2,2,2020-11-01,Camp de Mestalla,Valencia,Figueroa Vázquez,VALENCIA


In [14]:
"""
Preparamos un dataframe que unifique todos los datos de mediciones recogidas en pasos previos. Además se realizan
los ajustes necesarios para que el merge con los datos de partidos se correcto
"""

# Recogemos hora de inicio del proceso
t0 = time.time()

# Inicializamos una lista con los años de los cuales tenemos datos para unificar en un solo dataframe todos 
# los datos de mediciones que tenemos en ficheros JSON con nombres 'mediciones_YYYY.json'
años = list(range(2000,2021))

# Inicializamos lista vacia que contendra los dataframes generados para cada año. Esta valiable se utilizará
# después para concatenar todos los datos en un único df resultante
l_months = []

# Recorremos la lista de años y para cada año repetimos la operación:
#    1.- Leemos archivo json que volcamos en un diccionario
#    2.- Recorremos el diccionario:
#        a.- Generamos un df auxiliar que contiene los datos del item tratado (mediciones del mes)
#        b.- Añadimos a la lista l_months el df generado en el paso previo

for año in años:
        
    # 1.- Leemos el archivo json como dicionario
    with open('mediciones_'+str(año)+'.json') as json_data:
        data = json.load(json_data)
    
    # Recuperamos en una lista las claves de los items que deberemos tratar en cada iteración    
    l_items = data.keys()

    # 2.-
    for item in l_items:
        for month in data[item]:
            # 2.a.- Generamos df y lo trasponermos para corregir su estrcutura
            df1 = pd.DataFrame(month.items()).set_index(0).T
            # 2.b.
            l_months.append(df1)

# Concatenamos en un df resultante todos los datos (datos desde 2000-01-01 hasta 2020-11-xx, 
# depende de los datos disponibles en AEMet)
dfClima = pd.concat(l_months)
# Eliminamos la columna 'nombre' que ya no es de utilidad
dfClima.drop(['nombre'], axis = 'columns', inplace=True)
# Reemplazamos las ',' por '.' en los valores de tipo string que representan valores 'float' con el fin de
# convertirlos en floats
dfClima['tmed'] = dfClima['tmed'].str.replace(',','.').astype(float)
dfClima['prec'] = dfClima['prec'].str.replace(',','.')
dfClima['prec'] = dfClima['prec'].str.replace('Ip','0.0')
dfClima['prec'] = dfClima['prec'].str.replace('Acum','0.0')
dfClima['prec'] = dfClima['prec'].astype(float)

# Cambiamos el formato de la columna Fecha a formato fecha para poder ordenar los datos por dicha 
# columna de forma ascendente
dfClima['fecha'] = pd.to_datetime(dfClima['fecha'], format='%Y-%m-%d')

# 6.- Agrupamos los datos por fecha y provincia para evitar duplicados, haciendo media de los valores de tempreatura
#     y precipitaciones
test_df2 = dfClima.groupby(['fecha','provincia']).mean()

# Recogemos el tiempo invertido en el proceso 
response_delay = time.time() - t0

print("Done. Elapsed Time: "+str(response_delay))

2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


NameError: name 'response_delay' is not defined

In [15]:
"""
Dado que la agrupación omite repeticiones en las columnas de grupo, generamos un nuevo df con valores en todas 
las filas
"""
dfClima_res = test_df2.reset_index()

print(dfClima_res)

tmed  prec
fecha      provincia                   
2000-01-01 A CORUÑA      6.457143  0.00
           ALBACETE      3.250000  0.00
           ALICANTE      8.000000  0.00
           ALMERIA      10.600000  0.00
           ARABA/ALAVA   4.900000  0.00
...                           ...   ...
2020-11-03 TOLEDO       15.180000  1.24
           VALENCIA     16.171429  6.40
           VALLADOLID    9.600000  0.00
           ZAMORA       10.100000  0.00
           ZARAGOZA     11.480000  1.12

[394246 rows x 2 columns]

In [4]:
"""
Unimos los dataframes con datos de partido y clima, por Fecha y Provincia obteniendo como resultado un df completo
"""

resultado = pd.merge(df_Res_date, dfClima_res, left_on=["Fecha","Provincia"], right_on=["fecha","provincia"], how='left')
resultado.drop(['fecha','provincia'], axis = 'columns', inplace=True)

resultado.head()

NameError: name 'df_Res_date' is not defined

In [41]:
"""
Comprobaciones que podemos realizar antes de generar el csv final
"""
# Comprobamos la existencia de partidos sin datos climatológicos asociados
# resultado[resultado['tmed'].isnull()]

# Comprobamos que el número de registros generado coincide que el inicialmente presente en el csv partidos.csv
print("#rows_resultado: "+ str(len(resultado.index))+ " vs #rows_dsFutbol: " + str(len(dsFutbol.index)))

#rows_resultado: 7673 vs #rows_dsFutbol: 7673


In [1]:
resultado.to_csv('../result/stats_and_clima_1div_es_2000_2020.csv')

NameError: name 'resultado' is not defined